In [ ]:
from torch import LongTensor, FloatTensor
import torch
import math

In [ ]:
class Module(object):
                
    
    def forward(self , *x):
        raise NotImplementedError
    
    def backward(self , *gradwrtoutput):
        raise NotImplementedError
    
    def param(self):
        return []

In [ ]:
class Linear(Module):
    def __init__(self, in_features, out_features):
        super(Linear, self).__init__()
        self.weight = FloatTensor(out_features, in_features)
        self.bias = FloatTensor(out_features)
        self.reset_parameters()
        self.bias_grad = FloatTensor(self.bias.size()).zero_()
        self.weight_grad = FloatTensor(self.weight.size()).zero_()
        self.previous_x = None
        
        
    def reset_parameters(self):
        std = 1 / math.sqrt(self.weight.size(1))
        self.weight.uniform_(-std, std)
        self.bias.uniform_(-std, std)
    
    def forward(self, x):
        self.previous_x = x
        return x.matmul(self.weight.t()) + self.bias
    
    def backward(self, *gradwrtoutput):
        print(gradwrtoutput[0].size(), self.bias_grad.size(), self.previous_x.size())
        self.bias_grad.add_(gradwrtoutput[0].sum(0))
        self.weight_grad.add_(gradwrtoutput[0].matmul(self.previous_x).sum(0))
        return gradwrtoutput[0].matmul(self.weight.t())
        
    def param(self):
        return [(self.weight, self.weight_grad), (self.bias, self.bias_grad)]
        

In [ ]:
class Sequential(Module):
    def __init__(self, *args):
        self.module_array = []
        super(Sequential, self).__init__()
        for arg in args:
            self.module_array.append(arg)
    
    def forward(self, *x):
        input_ = x[0]
        for module in self.module_array:
            
            input_ = module.forward(input_)
        return input_
    
    def backward(self, *gradwrtoutput):
        input_ = gradwrtoutput[0]
        for module in self.module_array[::-1]:
            input_ = module.backward(input_)
            
        return input_
    
    def param(self):
        parameters = []
        for module in self.module_array:
            parameters.extend(module.param())
        return parameters

In [ ]:
class ReLU(Module):
    def __init__(self):
        super(ReLU, self).__init__()
        self.temp = None
    
    def forward(self, x):
        x[x<=0] = 0
        self.temp = x
        return x
    
    def backward(self, *gradwrtoutput):
        return self.dRelu(self.temp) * gradwrtoutput[0]
    
    def dRelu(self, x):
        x[x>0] = 1
        x[x<0] = 0
        return x
        

class Tanh(Module):
    
    def __init__(self):
        super(Tanh, self).__init__()
    
    def forward(self, x):
        return torch.tanh(x)
    
class MSELoss(Module):
    
    def __init__(self):
        super(MSELoss, self).__init__()
        
    def forward(self, *x):
        return (x[0] - x[1]).pow(2).sum()
    
    def backward(self, *gradwrtoutput):
        return 2 * (gradwrtoutput[1] - gradwrtoutput[0])

In [ ]:
model = Sequential(Linear(2,3),ReLU(), Linear(3,2))
loss = MSELoss()


input_ = torch.randn(4,2)
target = LongTensor([0,1,0,1])
output = model.forward(input_)
prediction = torch.max(output,1)[1]
prediction
grad = loss.backward(prediction, target)
grad
grad = model.backward(grad.float())
#model.parameters

#print(input_, target)


In [ ]:
model =
for i in range(200):
    for b in range(0, train_input.size(0), mini_batch_size):
        output =
        
        